In [1]:
import uuid
from typing import List, Dict, Any
import os
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Configure Gemini API
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

# Global generation config
GENERATION_CONFIG = {
    "temperature": 1,
    "top_k": 64,
    "max_output_tokens": 1000,
    "response_mime_type": "text/plain",
}

class Agent:
    def __init__(self, name: str, system_instruction: str):
        self.id = str(uuid.uuid4())
        self.name = name
        self.system_instruction = system_instruction
        self.memory: List[Dict[str, Any]] = [] # Memory should be overridden in the subclass

    # TO BE OVERRIDDEN
    def process_message(self, message: Dict[str, Any]) -> Dict[str, Any]:
        raise NotImplementedError("process_message method must be implemented in the subclass")

    # def receive_message(self, message: Dict[str, Any]):
    #     self.memory.append(message)
    #     return self.process_message(message)

    def send_message(self, recipient: 'Agent', content: str):
        message = {
            "sender": self.name,
            "recipient": recipient.name,
            "content": content
        }
        return recipient.process_message(message)

class GeminiAgent(Agent):
    def __init__(self, name: str, system_instruction: str):
        super().__init__(name, system_instruction)
        self.model = self.create_gemini_model(system_instruction=system_instruction)
        self.chat_session = self.model.start_chat(history=[])
        self.memory = self.chat_session.history

    def create_gemini_model(self, system_instruction: str = ""):
        return genai.GenerativeModel(
            model_name="gemini-1.5-flash",
            generation_config=GENERATION_CONFIG,
            safety_settings={
                HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
            },
            system_instruction=system_instruction,
        )
    
    def process_message(self, message: Dict[str, Any]) -> Dict[str, Any]:
        # Append user message to history
        self.chat_session.history.append({
            "role": "user",
            "parts": [message['content']],
        })
        
        response = self.chat_session.send_message(message['content'])
        
        # Append model response to history
        self.chat_session.history.append({
            "role": "model",
            "parts": [response.text],
        })
        
        return {"sender": self.name, "content": response.text}

class UserAgent(Agent):
    def __init__(self, name: str, system_instruction: str):
        super().__init__(name, system_instruction)
        self.memory = []

    def process_message(self, message: Dict[str, Any]) -> Dict[str, Any]:
        # Append user message to history
        print("Do nothing for now")
    
    def send_message(self, recipient: 'Agent', content: str):
        input_content = input(content)
        print("[user]",input_content)
        message = {
            "sender": self.name,
            "recipient": recipient.name,
            "content": input_content
        }
        # Append user message to memory
        self.memory.append({
            "role": "user",
            "parts": [input_content],
        })
        return recipient.process_message(message)

class Mediator:
    def __init__(self):
        self.agents: Dict[str, Agent] = {}
        self.agents["user"] = UserAgent("User", "") # Add an user agent as a default agent

    def add_agent(self, agent: Agent):
        self.agents[agent.id] = agent

    def get_agent(self, agent_id: str) -> Agent:
        return self.agents.get(agent_id)

    def communication(self, sender_id: str, recipient_id: str, content: str):
        sender = self.get_agent(sender_id)
        recipient = self.get_agent(recipient_id)
        if sender and recipient:
            return sender.send_message(recipient, content)
        else:
            raise ValueError("Invalid sender or recipient ID")
    
    def conversation(self, sender_id: str, recipient_id: str, content: str, str_condition: str = "", max_turns: int = 4):
        sender = self.get_agent(sender_id)
        recipient = self.get_agent(recipient_id)
        counter = 0
        res_recipient = content

        # Loop for up to 4 communications or until the condition is met
        while counter < max_turns:
            res_recipient = self.communication(sender_id, recipient_id, res_recipient['content'] if counter == 0 else res_sender['content'])
            print(f'[{recipient.name}]', res_recipient['content'])
            if str_condition and str_condition in res_recipient['content']:
                break
            res_sender = self.communication(recipient_id, sender_id, res_recipient['content'])
            print(f'[{sender.name}]', res_sender['content'])
            if str_condition and str_condition in res_sender['content']:
                break
            counter += 1
        
        return res_sender
    
    def user_conversation(self, sender_id: str, recipient_id: str, hint: str="Write your message", str_condition: str = "",  max_turns: int = 4):
        sender = self.get_agent(sender_id)
        res_sender = self.communication("user", sender_id, hint)
        print(f'[{sender.name}]', res_sender['content'])
        res = self.conversation(sender_id, recipient_id, content=res_sender, str_condition=str_condition, max_turns=4)
        return res



# class Orchestrator:
#     #TODO


# Create Mediator
Mediator = Mediator()

# Create Writer Agent
writer_agent = GeminiAgent("Writer", "You are a writer that write poems in max of 10 words. You just answer with poems.\n")
Mediator.add_agent(writer_agent)

# Create Critic Agent
critic_agent = GeminiAgent("Critic", "You are a critic that evaluates text. Give feedback on the text written by the writer in order to make the text better, be short with your responses.")
Mediator.add_agent(critic_agent)

# Create Language checker Agent
translator_agent = GeminiAgent("Language Checker", "You are a language checker that makes sure the text is written in Italian. Answer with just 'yes its ok, repeat you message' or 'no, please write it in italian'")
Mediator.add_agent(translator_agent)

# Create User Agent
user_agent = UserAgent("User", "")
Mediator.add_agent(user_agent)

# Start communication
print("### Start communication")
print("### Writer -> Critic conversation")
res1 = Mediator.user_conversation(writer_agent.id, critic_agent.id, hint="Tell me what you want to write", max_turns=4)
print("### Writer -> Language Checker conversation")
print("-" * 50)  # Prints a line of 50 dashes as a divider
res2 = Mediator.conversation(writer_agent.id, translator_agent.id, content=res1,str_condition="yes its ok", max_turns=8)

### Start communication
### Writer -> Critic conversation
[user] nature
[Writer] Green leaves whisper, wind's soft sigh. 

[Critic] This is a nice start, but it could use more detail.  Try adding a specific type of leaf or a stronger image of the wind. 

[Writer] Oak leaves rustle, wind's fierce dance. 

[Critic] This is much stronger!  The imagery is clearer and the "fierce dance" is evocative. 

[Writer] Sun-dappled forest, hushed and still. 

[Critic] This is a good visual, but it lacks a sense of movement.  Try adding a verb that evokes the forest's stillness. 

[Writer] Sunlight sleeps, forest holds its breath. 

[Critic] This is very effective! The personification of sunlight and the forest creates a vivid image of stillness. 

[Writer] Stream gurgles, mossy stones gleam. 

### Writer -> Language Checker conversation
--------------------------------------------------
[Language Checker] no, please write it in italian 

[Writer] Ruscello gorgoglia, pietre muschiose brillano. 

[Lan